# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [17]:
# Write your code below.
import os
import time
# set working dir to same as env file
os.chdir('../../01_materials/labs/')


%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [22]:
import os
from glob import glob

# Write your code below.
ft_dir = os.getenv("PRICE_DATA")
parq_file = glob(os.path.join(ft_dir, "*/*.parquet"))
df = dd.read_parquet(parq_file).compute().reset_index()



In [23]:
df

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year
0,0,2013-12-02 00:00:00-05:00,34.995835,35.126731,34.760216,34.825665,2039962,0.0,0.0,A,2013
1,1,2013-12-03 00:00:00-05:00,34.635851,34.799476,34.400232,34.642395,3462706,0.0,0.0,A,2013
2,2,2013-12-04 00:00:00-05:00,34.583503,35.231455,34.504966,35.067829,3377288,0.0,0.0,A,2013
3,3,2013-12-05 00:00:00-05:00,34.917279,35.211801,34.766746,35.015453,2530939,0.0,0.0,A,2013
4,4,2013-12-06 00:00:00-05:00,35.185648,35.951404,35.185648,35.885956,4268513,0.0,0.0,A,2013
...,...,...,...,...,...,...,...,...,...,...,...
118438,2553,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,YUM,2024
118439,2554,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,YUM,2024
118440,2555,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,YUM,2024
118441,2556,2024-01-30 00:00:00-05:00,129.076550,129.928083,128.304230,129.333984,1421500,0.0,0.0,YUM,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [24]:
# Write your code below.

import numpy as np
dd_feat = (df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


C:\Users\klawa\AppData\Local\Temp\ipykernel_2544\1595277154.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dd_feat = (df.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [32]:
dd_feat

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year,Close_lag,returns,hi_lo_range
0,0,2013-12-02 00:00:00-05:00,34.995835,35.126731,34.760216,34.825665,2039962,0.0,0.0,A,2013,NaN,NaN,0.366515
1,1,2013-12-03 00:00:00-05:00,34.635851,34.799476,34.400232,34.642395,3462706,0.0,0.0,A,2013,34.825665,-0.005262,0.399244
2,2,2013-12-04 00:00:00-05:00,34.583503,35.231455,34.504966,35.067829,3377288,0.0,0.0,A,2013,34.642395,0.012281,0.726489
3,3,2013-12-05 00:00:00-05:00,34.917279,35.211801,34.766746,35.015453,2530939,0.0,0.0,A,2013,35.067829,-0.001494,0.445055
4,4,2013-12-06 00:00:00-05:00,35.185648,35.951404,35.185648,35.885956,4268513,0.0,0.0,A,2013,35.015453,0.024861,0.765756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118438,2553,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,YUM,2024,128.165634,0.006026,1.752578
118439,2554,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,YUM,2024,128.937943,-0.008678,2.000117
118440,2555,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,YUM,2024,127.819061,0.011310,2.237759
118441,2556,2024-01-30 00:00:00-05:00,129.076550,129.928083,128.304230,129.333984,1421500,0.0,0.0,YUM,2024,129.264694,0.000536,1.623853


In [40]:
# Write your code below.
# Loading the necessary packages
import pandas as pd

# Convert dask dataframe to pandas dataframe
#pd_feat = dd_feat.compute()

# Calculate rolling average with a window of 10 days

pd_feat1 = dd_feat.groupby('ticker', group_keys=False).apply (
    lambda x: x.assign(rolling_avg_rtn=x['returns'].rolling(10).mean())
)

C:\Users\klawa\AppData\Local\Temp\ipykernel_2544\1539705068.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pd_feat1 = dd_feat.groupby('ticker', group_keys=False).apply (


In [41]:
pd_feat1

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year,Close_lag,returns,hi_lo_range,rolling_avg_rtn
0,0,2013-12-02 00:00:00-05:00,34.995835,35.126731,34.760216,34.825665,2039962,0.0,0.0,A,2013,NaN,NaN,0.366515,NaN
1,1,2013-12-03 00:00:00-05:00,34.635851,34.799476,34.400232,34.642395,3462706,0.0,0.0,A,2013,34.825665,-0.005262,0.399244,NaN
2,2,2013-12-04 00:00:00-05:00,34.583503,35.231455,34.504966,35.067829,3377288,0.0,0.0,A,2013,34.642395,0.012281,0.726489,NaN
3,3,2013-12-05 00:00:00-05:00,34.917279,35.211801,34.766746,35.015453,2530939,0.0,0.0,A,2013,35.067829,-0.001494,0.445055,NaN
4,4,2013-12-06 00:00:00-05:00,35.185648,35.951404,35.185648,35.885956,4268513,0.0,0.0,A,2013,35.015453,0.024861,0.765756,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118438,2553,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,YUM,2024,128.165634,0.006026,1.752578,0.000828
118439,2554,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,YUM,2024,128.937943,-0.008678,2.000117,0.000115
118440,2555,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,YUM,2024,127.819061,0.011310,2.237759,0.001145
118441,2556,2024-01-30 00:00:00-05:00,129.076550,129.928083,128.304230,129.333984,1421500,0.0,0.0,YUM,2024,129.264694,0.000536,1.623853,0.000672


: 

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.